In [ ]:
## Clone the Github Repo

import shutil
import os, sys
import json

shutil.rmtree('/content/cen_sierra_pywr_new', ignore_errors=True)
!git clone https://github.com/Maburidi/cen_sierra_pywr_new.git
sys.path.insert(0,'/content/cen_sierra_pywr_new/')

%cd /content/cen_sierra_pywr_new/


Cloning into 'cen_sierra_pywr_new'...
remote: Enumerating objects: 16502, done.
remote: Counting objects: 100% (1110/1110), done.
remote: Compressing objects: 100% (603/603), done.
remote: Total 16502 (delta 962), reused 507 (delta 507), pack-reused 15392 (from 5)
Receiving objects: 100% (16502/16502), 1012.99 MiB | 14.92 MiB/s, done.
Resolving deltas: 100% (7573/7573), done.
Updating files: 100% (1781/1781), done.
/content/cen_sierra_pywr_new


In [ ]:
# Install Pywr

!bash install_dependencies.sh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 30.9 MB/s eta 0:00:00


In [ ]:
# Run the Models


################## RESULTS ######################

# A folder called "results" will be created, where the results will be stored. Similar thing for loggs

################# Arguments #####################

# -b: Basin of choice, ['stanislaus', 'tuolumne', 'merced', 'upper_san_joaquin']
# -s : starting year: gcms: 2006  , loca2_gcms: 2015      Livenh: 1990
# -e : end year:      gcms: 2099  , loca2_gcms: 2099      Livenh: 2012
# -p: include planning: [0,1] for stanislaus, 0 for the rest
# -n: runname
# -d: Debug mode: [0,1], 1 to see more details for development
# -sc: Scenario set. ['debug','natural_flow', 'gcms', 'loca2_gcms']
#         Scenario working so far: 1- 'debug'
#                                  2- 'natural_flow'
#                                  3- 'gcms' --- only: tuolumne CCSM4_rcp85, stanislaus: ACCESS1_0_rcp85
#                                  4- 'loca2_gcms' --- only: ACCESS_CM2 for tuolumne and stanislaus
# -gm: gcms models, if its   ['CCSM4_rcp85','ACCESS1_0_rcp85]
# -lgm: LOCA2 gcms models    [ACCESS_CM2, ]

#======= CL arguments for Planning mode ---
# -m: Planning months, default 8,
# -bl: Number of piecewise blocks, default 5




################# RUN SCENARIOS ###################

#Examples: uncomment the command line to run the scenario you prefer:

#-----------------
#[A] LOCA2 GCMS Dataset
#!python main.py -b 'stanislaus' -s 2015 -e 2099 -d 0 -p 1 -sc 'loca2_gcms' -lgm 'ACCESS_CM2'
!python main.py -b 'tuolumne' -s 2015 -e 2099 -d 1 -sc 'loca2_gcms' -lgm 'ACCESS_CM2'
#!python main.py -b 'upper_san_joaquin' -s 2015 -e 2099 -d 0 -p 1 -sc 'loca2_gcms' -lgm 'ACCESS_CM2' -m 2 -bl 2
#!python main.py -b 'merced' -s 2015 -e 2099 -d 1 -sc 'loca2_gcms' -lgm 'ACCESS_CM2'                          # still getting an error

#-----------------
#[B] GCMS Dataset
#!python main.py -b 'stanislaus' -s 2006 -e 2099 -d 1 -p 1 -sc 'gcms' -gm 'ACCESS1_0_rcp85'
#!python main.py -b 'tuolumne' -s 2006 -e 2099 -d 1 -sc 'gcms' -gm 'CCSM4_rcp85'

#-----------------
#[C] Livenh Historical natural_flow
#!python main.py -b 'tuolumne' -s 1990 -e 2012 -d 0 -sc 'natural_flow'
#!python main.py -b 'merced' -s 1990 -e 2012 -d 1 -sc 'natural_flow'                                         # still getting an error
#!python main.py -b 'stanislaus' -s 1990 -e 2012 -d 0 -p 1 -sc 'natural_flow'
#!python main.py -b 'upper_san_joaquin' -s 1990 -e 2012 -d 1 -p 1 -sc 'natural_flow'




2025-02-04 23:41:17.202 | INFO     | sierra.run_basin_model:_run_model:79 - Running "Natural Flow" scenario for TUOLUMNE basin, HISTORICAL/LIVNEH climate
2025-02-04 23:41:17.212 | INFO     | sierra.run_basin_model:_run_model:227 - Parameters Imported
2025-02-04 23:41:17.221 | INFO     | sierra.run_basin_model:_run_model:244 - tuolumne Policy Parameters Imported
2025-02-04 23:41:17.224 | INFO     | sierra.run_basin_model:_run_model:356 - Loading daily model
2025-02-04 23:41:17.734 | INFO     | sierra.run_basin_model:_run_model:362 - Daily Model Loaded Successfully
2025-02-04 23:41:17.735 | INFO     | sierra.run_basin_model:_run_model:364 - Setup Daily Model
/usr/local/lib/python3.11/dist-packages/pywr/dataframe_tools.py:127: FutureWarning: Resampling with a PeriodIndex is deprecated. Cast index to DatetimeIndex before resampling instead.
  new_df = new_df.resample(target_index.freq).agg(resample_func)
/usr/local/lib/python3.11/dist-packages/pywr/dataframe_tools.py:140: FutureWarning: Re